In [1]:
from google.colab import drive
drive.mount('/content/drive')

ROOT_DIR = "/content/drive/MyDrive/firstaid_rag_project"
print("ROOT_DIR:", ROOT_DIR)


Mounted at /content/drive
ROOT_DIR: /content/drive/MyDrive/firstaid_rag_project


In [2]:
!pip install -q transformers accelerate


In [3]:
from sentence_transformers import SentenceTransformer

# Embedding model for FAISS
emb_model = SentenceTransformer("BAAI/bge-small-en-v1.5")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

# Tiny, fast LLM (1.1B params)
LLM_NAME = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

llm_tokenizer = AutoTokenizer.from_pretrained(LLM_NAME)

llm_model = AutoModelForCausalLM.from_pretrained(
    LLM_NAME,
    device_map="auto",
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
)

llm_pipe = pipeline(
    "text-generation",
    model=llm_model,
    tokenizer=llm_tokenizer,
    max_new_tokens=128,          # smaller output = faster
    temperature=0.2,
    top_p=0.9,
    do_sample=True,
    return_full_text=False,
)


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Device set to use cpu


In [5]:
!pip install -q faiss-cpu sentence-transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 21.6 MB/s eta 0:00:00


In [6]:
import os
import pickle
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer

VECTOR_DIR = os.path.join(ROOT_DIR, "vector_store")
INDEX_PATH = os.path.join(VECTOR_DIR, "first_aid_index.faiss")
DOCS_PATH  = os.path.join(VECTOR_DIR, "first_aid_docs.pkl")

print("INDEX_PATH:", INDEX_PATH)
print("DOCS_PATH :", DOCS_PATH)

index = faiss.read_index(INDEX_PATH)
print("Loaded FAISS index with dimension:", index.d)

with open(DOCS_PATH, "rb") as f:
    all_docs = pickle.load(f)
print("Loaded docs:", len(all_docs))

model = SentenceTransformer("BAAI/bge-small-en-v1.5")
print("Model loaded.")


INDEX_PATH: /content/drive/MyDrive/firstaid_rag_project/vector_store/first_aid_index.faiss
DOCS_PATH : /content/drive/MyDrive/firstaid_rag_project/vector_store/first_aid_docs.pkl
Loaded FAISS index with dimension: 384
Loaded docs: 43
Model loaded.


In [7]:
def embed_query(question: str) -> np.ndarray:
    q_emb = model.encode(
        question,
        convert_to_numpy=True
    ).astype("float32")
    faiss.normalize_L2(q_emb.reshape(1, -1))
    return q_emb.reshape(1, -1)

def retrieve(question: str, k: int = 3):
    q_emb = embed_query(question)
    D, I = index.search(q_emb, k)
    results = []
    for score, idx in zip(D[0], I[0]):
        if idx == -1:
            continue
        results.append((all_docs[idx], float(score)))
    return results

def print_results(question: str, k: int = 3):
    print("QUESTION:", question)
    res = retrieve(question, k=k)
    for i, (doc, score) in enumerate(res, start=1):
        print("-" * 80)
        print(f"RESULT {i} (score={score:.3f})")

        # Ensure we have a string to display
        if isinstance(doc, str):
            text = doc
        else:
            # fall back: convert dict / row / anything else to string
            text = str(doc)

        print(text[:1000])   # show only first 1000 chars so it’s not huge
    print("-" * 80)


def answer_question(question: str, k: int = 3) -> None:
    """
    Retrieve top-k documents and print a user-friendly first-aid answer.
    """
    results = retrieve(question, k=k)

    if not results:
        print("I couldn't find anything for that question in the first-aid guide.")
        return

    print("QUESTION:", question)
    print("-" * 80)

    # Use the top result as the main answer
    top_doc, top_score = results[0]

    # Safely get the text field
    if isinstance(top_doc, dict) and "text" in top_doc:
        main_text = top_doc["text"]
        source_id = top_doc.get("id", "unknown")
    else:
        main_text = str(top_doc)
        source_id = "unknown"

    print(f"Best match (score={top_score:.3f}, source={source_id})")
    print("-" * 80)
    print(main_text[:1500])   # show up to ~1500 chars of the main guidance
    print("-" * 80)

    # Optionally show other supporting sources briefly
    if len(results) > 1:
        print("Other supporting sources:")
        for (doc, score) in results[1:]:
            if isinstance(doc, dict):
                sid = doc.get("id", "unknown")
            else:
                sid = "unknown"
            print(f"  - {sid} (score={score:.3f})")

    # Safety disclaimer
    print()
    print("⚠️  Reminder: This is a basic first-aid guide, not professional medical advice.")
    print("    For serious symptoms or emergencies, call local emergency services.")




In [8]:
print_results("What should I do for a minor cut?")


QUESTION: What should I do for a minor cut?
--------------------------------------------------------------------------------
RESULT 1 (score=0.796)
{'id': 'CSV::CUT_MINOR', 'text': '[CONDITION_ID: CUT_MINOR]\nCondition Name: Minor cut or scrape\nCategory: Injury\nSeverity: mild\n\nCommon Signs:\nSmall, shallow cut; light bleeding; skin surface only; person awake and breathing normally.\n\nImmediate First-Aid Steps:\nWash your hands if possible. Gently rinse the area with clean running water. If needed, clean around (not inside) the wound with mild soap. Apply gentle pressure with a clean cloth until bleeding slows or stops. Cover with a clean bandage or dressing.\n\nWhat NOT To Do:\nDo not use harsh chemicals directly in the wound. Do not pick at scabs. Do not ignore increasing redness, warmth, swelling, or pus.\n\nWhen To Seek Professional Help:\nIf bleeding does not stop after about 10–15 minutes of firm pressure, if the cut is deep, gaping, on the face, caused by a very dirty or rus

In [9]:
print_results("What should I do for a minor burn?")


QUESTION: What should I do for a minor burn?
--------------------------------------------------------------------------------
RESULT 1 (score=0.791)
{'id': 'CSV::BURN_MILD', 'text': '[CONDITION_ID: BURN_MILD]\nCondition Name: Mild burn (no blisters)\nCategory: Injury\nSeverity: mild\n\nCommon Signs:\nRed, painful skin in a small area; top layer of skin affected; no blisters; usually from brief contact with a hot object, liquid, or steam.\n\nImmediate First-Aid Steps:\nMove away from the heat source. Cool the area with cool (not ice-cold) running water or a cool, wet cloth for about 10–20 minutes or until pain improves. Gently remove rings or tight items near the burn if possible. After cooling, cover lightly with a clean, non-stick dressing.\n\nWhat NOT To Do:\nDo not apply ice directly to the burned skin. Do not use butter, oils, or greasy home remedies. Do not break any blisters that appear later.\n\nWhen To Seek Professional Help:\nIf the burn is larger than the person’s palm, on th

In [10]:
answer_question("What should I do for a minor cut?")


QUESTION: What should I do for a minor cut?
--------------------------------------------------------------------------------
Best match (score=0.796, source=CSV::CUT_MINOR)
--------------------------------------------------------------------------------
[CONDITION_ID: CUT_MINOR]
Condition Name: Minor cut or scrape
Category: Injury
Severity: mild

Common Signs:
Small, shallow cut; light bleeding; skin surface only; person awake and breathing normally.

Immediate First-Aid Steps:
Wash your hands if possible. Gently rinse the area with clean running water. If needed, clean around (not inside) the wound with mild soap. Apply gentle pressure with a clean cloth until bleeding slows or stops. Cover with a clean bandage or dressing.

What NOT To Do:
Do not use harsh chemicals directly in the wound. Do not pick at scabs. Do not ignore increasing redness, warmth, swelling, or pus.

When To Seek Professional Help:
If bleeding does not stop after about 10–15 minutes of firm pressure, if the cut is

In [11]:
answer_question("How should I treat a mild burn?")


QUESTION: How should I treat a mild burn?
--------------------------------------------------------------------------------
Best match (score=0.835, source=CSV::BURN_MILD)
--------------------------------------------------------------------------------
[CONDITION_ID: BURN_MILD]
Condition Name: Mild burn (no blisters)
Category: Injury
Severity: mild

Common Signs:
Red, painful skin in a small area; top layer of skin affected; no blisters; usually from brief contact with a hot object, liquid, or steam.

Immediate First-Aid Steps:
Move away from the heat source. Cool the area with cool (not ice-cold) running water or a cool, wet cloth for about 10–20 minutes or until pain improves. Gently remove rings or tight items near the burn if possible. After cooling, cover lightly with a clean, non-stick dressing.

What NOT To Do:
Do not apply ice directly to the burned skin. Do not use butter, oils, or greasy home remedies. Do not break any blisters that appear later.

When To Seek Professional Hel

In [12]:
# 1. Some test questions + which condition we expect on top
test_queries = [
    ("What should I do for a minor cut?", "CSV::CUT_MINOR"),
    ("How should I treat a mild burn?", "CSV::BURN_MILD"),
    ("What should I do for a nosebleed?", "CSV::NOSEBLEED_SIMPLE"),
    ("How do I care for a sprained ankle?", "CSV::SPRAIN_ANKLE"),
    ("What should I do if someone faints?", None),  # PDF/FAQ mostly
]

# 2. Tiny evaluator
def evaluate_queries(queries, k=3):
    total = 0
    correct_top1 = 0
    correct_topk = 0

    for question, expected in queries:
        total += 1
        results = retrieve(question, k=k)   # uses your existing retrieve()

        ids = []
        for doc, score in results:
            if isinstance(doc, dict):
                ids.append(doc.get("id", "unknown"))
            else:
                ids.append("unknown")

        print("QUESTION:", question)
        print("  expected:", expected)
        print("  got:", ids)

        if expected is not None:
            if ids and ids[0] == expected:
                correct_top1 += 1
            if expected in ids:
                correct_topk += 1
        print("-" * 60)

    labeled = sum(q[1] is not None for q in queries)
    print("\nSUMMARY")
    print("Total tested:", total)
    print("Labeled (have expected id):", labeled)
    if labeled > 0:
        print("Top-1 accuracy:", f"{correct_top1}/{labeled}")
        print(f"Top-{k} accuracy:", f"{correct_topk}/{labeled}")

# 3. Run it
evaluate_queries(test_queries, k=3)


QUESTION: What should I do for a minor cut?
  expected: CSV::CUT_MINOR
  got: ['CSV::CUT_MINOR', 'TXT::chunk_2', 'TXT::chunk_1']
------------------------------------------------------------
QUESTION: How should I treat a mild burn?
  expected: CSV::BURN_MILD
  got: ['CSV::BURN_MILD', 'PDF::page_3', 'CSV::SUNBURN_MILD']
------------------------------------------------------------
QUESTION: What should I do for a nosebleed?
  expected: CSV::NOSEBLEED_SIMPLE
  got: ['CSV::NOSEBLEED_SIMPLE', 'PDF::page_5', 'TXT::chunk_8']
------------------------------------------------------------
QUESTION: How do I care for a sprained ankle?
  expected: CSV::SPRAIN_ANKLE
  got: ['PDF::page_6', 'CSV::SPRAIN_ANKLE_MILD', 'TXT::chunk_8']
------------------------------------------------------------
QUESTION: What should I do if someone faints?
  expected: None
  got: ['CSV::FAINTING_BRIEF', 'PDF::page_9', 'PDF::page_8']
------------------------------------------------------------

SUMMARY
Total tested: 5
Lab

In [13]:
def interactive_demo():
    print("First-Aid RAG Assistant")
    print("Type 'quit' to stop.\n")
    while True:
        q = input("You: ")
        if q.strip().lower() in {"quit", "exit"}:
            print("Assistant: Take care! 👋")
            break
        print()
        answer_question(q)   # your existing function
        print("\n")


In [14]:
interactive_demo()


First-Aid RAG Assistant
Type 'quit' to stop.

You: quit
Assistant: Take care! 👋


In [15]:
import os, pickle
import faiss
import numpy as np

# Paths to your project
ROOT_DIR    = "/content/drive/MyDrive/firstaid_rag_project"
VECTOR_DIR  = os.path.join(ROOT_DIR, "vector_store")
INDEX_PATH  = os.path.join(VECTOR_DIR, "first_aid_index.faiss")
DOCS_PATH   = os.path.join(VECTOR_DIR, "first_aid_docs.pkl")

# Load FAISS index
index = faiss.read_index(INDEX_PATH)

# Load documents list
with open(DOCS_PATH, "rb") as f:
    all_docs = pickle.load(f)

# Split into ids and text so retrieve() can use them
doc_ids   = [d["id"] for d in all_docs]
documents = [d["text"] for d in all_docs]

print("Loaded docs:", len(documents))
print("First doc id:", doc_ids[0])
print("First doc preview:", documents[0][:120], "...")


Loaded docs: 43
First doc id: CSV::CUT_MINOR
First doc preview: [CONDITION_ID: CUT_MINOR]
Condition Name: Minor cut or scrape
Category: Injury
Severity: mild

Common Signs:
Small, shal ...


In [16]:
import numpy as np
import faiss

def embed_query(question: str) -> np.ndarray:
    """
    Encode a user question into a normalized embedding vector.

    Uses the same SentenceTransformer model you used when building the index.
    """
    # 1) Get embedding as 1D vector
    q_emb = emb_model.encode(question, convert_to_numpy=True).astype("float32")

    # 2) Reshape to (1, d) so FAISS is happy
    q_emb = q_emb.reshape(1, -1)

    # 3) Normalize L2 (same as when you built the index)
    faiss.normalize_L2(q_emb)

    return q_emb




def retrieve(question: str, k: int = 3):
    """
    Return top-k matches for a user question as a list of dicts:
    [
      {"id": doc_id, "score": 0.79, "text": "..."},
      ...
    ]
    """
    q_emb = embed_query(question)
    D, I = index.search(q_emb, k)  # D = scores, I = indices

    hits = []
    for score, idx in zip(D[0], I[0]):
        if idx == -1:
            continue
        doc_text = documents[idx]          # or however you stored them
        doc_id   = doc_ids[idx]            # e.g. "CSV::CUT_MINOR"
        hits.append({
            "id": doc_id,
            "score": float(score),
            "text": doc_text
        })
    return hits


In [17]:
hits = retrieve("What should I do for a minor cut?", k=3)
hits[0]


{'id': 'CSV::CUT_MINOR',
 'score': 0.7963465452194214,
 'text': '[CONDITION_ID: CUT_MINOR]\nCondition Name: Minor cut or scrape\nCategory: Injury\nSeverity: mild\n\nCommon Signs:\nSmall, shallow cut; light bleeding; skin surface only; person awake and breathing normally.\n\nImmediate First-Aid Steps:\nWash your hands if possible. Gently rinse the area with clean running water. If needed, clean around (not inside) the wound with mild soap. Apply gentle pressure with a clean cloth until bleeding slows or stops. Cover with a clean bandage or dressing.\n\nWhat NOT To Do:\nDo not use harsh chemicals directly in the wound. Do not pick at scabs. Do not ignore increasing redness, warmth, swelling, or pus.\n\nWhen To Seek Professional Help:\nIf bleeding does not stop after about 10–15 minutes of firm pressure, if the cut is deep, gaping, on the face, caused by a very dirty or rusty object, or if signs of infection appear.\n\nFollow-Up Advice:\nKeep the area clean and dry. Change the bandage if 

In [18]:
!pip install -q transformers accelerate sentencepiece

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

llm_model_name = "microsoft/Phi-3-mini-4k-instruct"  # or whichever you used

llm_tokenizer = AutoTokenizer.from_pretrained(llm_model_name)
llm_model = AutoModelForCausalLM.from_pretrained(
    llm_model_name,
    device_map="auto",
    torch_dtype="auto"
)

llm_pipe = pipeline(
    "text-generation",
    model=llm_model,
    tokenizer=llm_tokenizer,
    max_new_tokens=256,
    temperature=0.2,
    top_p=0.9,
    do_sample=True,
    return_full_text=False,
)

print("Loaded LLM:", LLM_NAME)


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

Device set to use cpu


Loaded LLM: TinyLlama/TinyLlama-1.1B-Chat-v1.0


In [19]:
SYSTEM_PROMPT = """You are a cautious first-aid assistant.
You MUST follow these rules:

- Answer ONLY using the provided context from first-aid resources.
- If the context does not clearly answer the question, say:
  "I’m not sure from these materials. Please contact a medical professional or emergency services."
- You are NOT a doctor and cannot give diagnoses.
- If there are any red-flag symptoms (trouble breathing, chest pain, confusion,
  severe bleeding, loss of consciousness, signs of stroke/heart attack, etc.),
  ALWAYS recommend calling local emergency services.

Keep answers short, clear, step-by-step, and easy for a non-medical person to follow.
ALWAYS mention that this is not a substitute for professional medical care.
"""

def build_prompt(question: str, hits):
    """Create a single text prompt that includes system instructions + retrieved context."""

    context_blocks = []
    for i, h in enumerate(hits, start=1):
        context_blocks.append(
            f"[SOURCE {i} | id={h['id']} | score={h['score']:.3f}]\n{h['text']}"
        )

    context_text = "\n\n".join(context_blocks)

    prompt = f"""{SYSTEM_PROMPT}

Context from first-aid resources:
{context_text}

User question:
{question}

Your answer (stay within the context above and be very cautious):"""

    return prompt


In [20]:
def answer_question(question: str, k: int = 3):
    """
    1. Use FAISS + embeddings to retrieve top-k relevant chunks.
    2. Build a grounded prompt from those chunks.
    3. Ask the local LLM to generate an answer.
    4. Return both the answer and the sources used.
    """
    # 1. Retrieve from your vector store
    hits = retrieve(question, k=k)

    # 2. Build prompt
    prompt = build_prompt(question, hits)

    # 3. Call local LLM (using the pipeline we created above)
    outputs = llm_pipe(
        prompt,
        pad_token_id=llm_tokenizer.eos_token_id,  # avoid warnings
    )

    answer_text = outputs[0]["generated_text"].strip()

    # 4. Package result with sources
    return {
        "question": question,
        "answer": answer_text,
        "sources": hits,
    }


In [ ]:
q1 = "What should I do for a minor burn?"
result1 = answer_question(q1, k=3)

print("QUESTION:", result1["question"])
print("\nANSWER:\n", result1["answer"])
print("\nSOURCES USED:")
for s in result1["sources"]:
    print(f"- {s['id']} (score={s['score']:.3f})")
